# 核心思路
top250电影的评论页面连接如下：
https://movie.douban.com/subject/6786002/comments?status=P

每个电影有其独有的代号（如6786002）

* 步骤：
1. 在top250页面(https://movie.douban.com/top250),先爬取250部电影的评论页面的url，存储在url.txt中后续备用
2. 读取url.txt，逐部电影进入其评论页面提取评论，存储在output.txt中


* 反爬措施：
1. 轮换headers中的user-agent
2. 设置time.sleep(random.random()*5)
3. 切换ip

* 文件介绍
1. output.txt:爬虫到的信息存储在本文件中
2. url.txt：爬取到的top250电影的评论页面url，存储在本文件中
3. request.ipynb：代码文件
4. need.txt：由于反反爬虫措施不够强力，爬虫过程中漏爬的评论页面url存储在本文件中，以备后续补充爬取


## 前置准备

In [1]:
"""
Crawler

@author: 陈梓轩
"""
import os
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
import cv2 
import requests
import time
import selenium
import requests
from bs4 import BeautifulSoup
import re
import random
import csv

# 输出评论的文件名
output_file = "test_data.csv"

# 中间文件名
need_file = "need.txt"

# 存储URL的文件名
url_file = 'urls.txt'

# request头部
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36 Edg/120.0.0.0'}

# user-agent的轮换列表
user_agent = [
    "Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
    "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:38.0) Gecko/20100101 Firefox/38.0",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; .NET4.0C; .NET4.0E; .NET CLR 2.0.50727; .NET CLR 3.0.30729; .NET CLR 3.5.30729; InfoPath.3; rv:11.0) like Gecko",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)",
    "Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0)",
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1)",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1",
    "Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/20100101 Firefox/4.0.1",
    "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; en) Presto/2.8.131 Version/11.11",
    "Opera/9.80 (Windows NT 6.1; U; en) Presto/2.8.131 Version/11.11",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_0) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Maxthon 2.0)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; TencentTraveler 4.0)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; The World)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SE 2.X MetaSr 1.0; SE 2.X MetaSr 1.0; .NET CLR 2.0.50727; SE 2.X MetaSr 1.0)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; 360SE)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Avant Browser)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)",
    "Mozilla/5.0 (iPhone; U; CPU iPhone OS 4_3_3 like Mac OS X; en-us) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8J2 Safari/6533.18.5",
    "Mozilla/5.0 (iPod; U; CPU iPhone OS 4_3_3 like Mac OS X; en-us) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8J2 Safari/6533.18.5",
    "Mozilla/5.0 (iPad; U; CPU OS 4_3_3 like Mac OS X; en-us) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8J2 Safari/6533.18.5",
    "Mozilla/5.0 (Linux; U; Android 2.3.7; en-us; Nexus One Build/FRF91) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1",
    "MQQBrowser/26 Mozilla/5.0 (Linux; U; Android 2.3.7; zh-cn; MB200 Build/GRJ22; CyanogenMod-7) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1",
    "Opera/9.80 (Android 2.3.4; Linux; Opera Mobi/build-1107180945; U; en-GB) Presto/2.8.149 Version/11.10",
    "Mozilla/5.0 (Linux; U; Android 3.0; en-us; Xoom Build/HRI39) AppleWebKit/534.13 (KHTML, like Gecko) Version/4.0 Safari/534.13",
    "Mozilla/5.0 (BlackBerry; U; BlackBerry 9800; en) AppleWebKit/534.1+ (KHTML, like Gecko) Version/6.0.0.337 Mobile Safari/534.1+",
    "Mozilla/5.0 (hp-tablet; Linux; hpwOS/3.0.0; U; en-US) AppleWebKit/534.6 (KHTML, like Gecko) wOSBrowser/233.70 Safari/534.6 TouchPad/1.0",
    "Mozilla/5.0 (SymbianOS/9.4; Series60/5.0 NokiaN97-1/20.0.019; Profile/MIDP-2.1 Configuration/CLDC-1.1) AppleWebKit/525 (KHTML, like Gecko) BrowserNG/7.1.18124",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows Phone OS 7.5; Trident/5.0; IEMobile/9.0; HTC; Titan)",
    "UCWEB7.0.2.37/28/999",
    "NOKIA5700/ UCWEB7.0.2.37/28/999",
    "Openwave/ UCWEB7.0.2.37/28/999",
    "Mozilla/4.0 (compatible; MSIE 6.0; ) Opera/UCWEB7.0.2.37/28/999",
]


## 爬取top250电影影评首页的url

In [10]:
base_url = 'https://movie.douban.com/top250'
params = {
    'filter': '',
}

movie_list=[]
for page in range(0, 250, 25):  # 从第一页到第五页，每页递增25
    params['start'] = page
    r = requests.get(base_url, params = params , headers = headers)
    soup = BeautifulSoup(r.text, 'lxml')

    movie_items = soup.find_all('li')
    for movie_item in movie_items:
        # movie_url
        if movie_item.find('div', class_='pic') is not None:
            movie_url = movie_item.find('a').get('href')
            movie_list.append(movie_url+'comments/')

# 存储URL的文件名
url_file = 'urls.txt'

# 将URL写入文件
with open(url_file, 'w', encoding='utf-8') as f:
    for url in movie_list:
        f.write(url + '\n')

## 爬取指定电影URL的评论数据

In [4]:
# 提取URL
with open(url_file, 'r', encoding='utf-8') as f:
    movie_list = [line.strip() for line in f.readlines()]

# 从250个电影评论url文件中逐个提取
i=1
for base_url in movie_list:

    print("正在爬取电影",i)
    i+=1
    
    params = {
        'limit': 20,
        'status': 'P',
        'sort': 'new_score',
        'percent_type': 'l'#评论类型
    }

    # 设置user-agent轮换
    headers = {
	'User-Agent':random.choice(user_agent)
    }

    symbol = True
    time.sleep(random.random()*5) # time.sleep 反反爬虫手段，并且可以留时间给动态页面加载
    for page in range(0, 200, 20):  # 从第一页到第五页，每页递增20

        params['start'] = page
        r = requests.get(base_url, params=params, headers = headers)

        time.sleep(random.random()*5) # time.sleep 反反爬虫手段，并且可以留时间给动态页面加载

        # 判断request是否收到正确响应
        if r.status_code == 200:
            # 继续解析和处理

            soup = BeautifulSoup(r.text, 'lxml')

            # 处理soup为空的情况
            if soup == None:
                with open(need_file, 'a', encoding='utf-8') as f:
                    f.write(base_url + '\n') 
                with open(output_file, 'a', encoding='utf-8') as f:
                    f.write("Unknown电影的soup为空")
                continue

            # 添加输出电影名
            if symbol==True:

                name = soup.find('div', id='content')
                if name is not None:
                    name = name.find('h1').text
                    # with open(output_file, 'a', encoding='utf-8') as csvfile:
                    #     writer = csv.writer(csvfile)
                    #     name_parts = name.split()
                    #     if name_parts:
                    #         row = [name_parts[0] + '的评论：']
                    #         writer.writerow(row)
                    symbol = False

                # 处理未找到元素的情况
                else:
                    # 记录下未爬取成功的url，后续重爬
                    with open(need_file, 'a', encoding='utf-8') as f:
                        f.write(base_url + '\n')
                    # with open(output_file, 'a', encoding='utf-8') as csvfile:
                    #     writer = csv.writer(csvfile)
                    #     row = [name_parts[0] + '的评论：']
                    #     writer.writerow(row)
                    time.sleep(0.5)
                    break


            # 输出所有评论，包括作者、星级、评论内容
                
            comment_items = soup.find_all('div',class_ = 'comment-item')

            for comment_item in comment_items:
                # username
                span = comment_item.find('span',class_ = 'comment-info')
                username = span.find('a').text

                # star
                stars_element = comment_item.find('span',class_ = re.compile(r'allstar.*rating'))
                if stars_element is not None:
                    stars_class = stars_element.get('class')
                    stars = stars_class[0].replace('allstar','')
                else:
                    stars = None

                # comment
                comment =  comment_item.find('span',class_ ="short").text
                comment = comment.replace('\n', ' ').replace('\r', ' ')

                # 将评论写入output.csv
                with open(output_file, 'a', encoding='utf-8') as csvfile:
                    writer = csv.writer(csvfile)
                    star = stars if stars else ""
                    row = [username, star, comment]
                    writer.writerow(row)


        else:
            print('请求失败:', r.status_code)


正在爬取电影 1
正在爬取电影 2
正在爬取电影 3
正在爬取电影 4
正在爬取电影 5
正在爬取电影 6
正在爬取电影 7
正在爬取电影 8
正在爬取电影 9
正在爬取电影 10
正在爬取电影 11


KeyboardInterrupt: 